У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [2]:
import pandas as pd

# Завантажуємо дані
df = pd.read_csv('/content/customer_segmentation_train.csv')

# Переглянемо перші рядки, щоб зрозуміти структуру
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


Цільова змінна: Segmentation

**🔧 Далі виконаємо:**

Заповнення пропущених значень.

Кодування категоріальних ознак.

Розділення на train/test (80/20).

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Копія для безпеки
data = df.copy()

# Заповнення пропущених значень
# - числові: на середнє
data['Work_Experience'].fillna(data['Work_Experience'].median(), inplace=True)
data['Family_Size'].fillna(data['Family_Size'].median(), inplace=True)

# - категоріальні: на найчастіше значення
for col in ['Profession', 'Var_1']:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Кодування категоріальних ознак
label_enc_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession',
                  'Spending_Score', 'Var_1', 'Segmentation']

encoders = {}
for col in label_enc_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    encoders[col] = le

# Розділення на ознаки та ціль
X = data.drop(['ID', 'Segmentation'], axis=1)
y = data['Segmentation']

# Тренувальна та тестова вибірка
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Збережемо у CSV для використання у Google Colab
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_path = '/content/preprocessed_train.csv'
test_path = '/content/preprocessed_test.csv'

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

(train_path, test_path)

/tmp/ipython-input-6-895811731.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Work_Experience'].fillna(data['Work_Experience'].median(), inplace=True)
/tmp/ipython-input-6-895811731.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].met

('/content/preprocessed_train.csv', '/content/preprocessed_test.csv')

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [8]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

# Для SMOTENC потрібно вказати індекси колонок, які є категоріальними
categorical_features = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
                        'Spending_Score', 'Var_1']
cat_feature_indices = [X.columns.get_loc(col) for col in categorical_features]

# Створимо SMOTENC
smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)

# SMOTE + Tomek
smote_tomek = SMOTETomek(smote=SMOTENC(categorical_features=cat_feature_indices, random_state=42), random_state=42)

# Виконаємо ресемплінг
X_train_smote, y_train_smote = smotenc.fit_resample(X_train, y_train)
X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train, y_train)

# Збережемо результати для Colab
pd.concat([X_train_smote, y_train_smote], axis=1).to_csv('/content/train_smote.csv', index=False)
pd.concat([X_train_smote_tomek, y_train_smote_tomek], axis=1).to_csv('/content/train_smote_tomek.csv', index=False)

('/content/train_smote.csv', '/content/train_smote_tomek.csv')

('/content/train_smote.csv', '/content/train_smote_tomek.csv')

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [10]:
# Встановлюємо бібліотеку для ресемплінгу
!pip install -q imbalanced-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

# ===========================
# 1. Завантаження підготовлених даних
# ===========================
# Завантажуємо дані, які ми попередньо створили (після кодування та заповнення пропусків)
train_df = pd.read_csv('/content/preprocessed_train.csv')
test_df = pd.read_csv('/content/preprocessed_test.csv')

# ===========================
# 2. Поділ на ознаки та ціль
# ===========================
X_train = train_df.drop('Segmentation', axis=1)
y_train = train_df['Segmentation']
X_test = test_df.drop('Segmentation', axis=1)
y_test = test_df['Segmentation']

# ===========================
# 3. Використання SMOTENC для врахування категоріальних ознак
# ===========================
# Створюємо список індексів колонок, які є категоріальними
categorical_features = ['Gender', 'Ever_Married', 'Graduated', 'Profession',
                        'Spending_Score', 'Var_1']
cat_feature_indices = [X_train.columns.get_loc(col) for col in categorical_features]

# Ініціалізуємо SMOTENC
smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)

# Застосовуємо SMOTE для збалансованих даних
X_train_smote, y_train_smote = smotenc.fit_resample(X_train, y_train)

# Застосовуємо SMOTE-Tomek для ще кращого ресемплінгу
smote_tomek = SMOTETomek(smote=SMOTENC(categorical_features=cat_feature_indices, random_state=42),
                         random_state=42)
X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train, y_train)

# ===========================
# 4. Навчання моделей One-vs-Rest Logistic Regression
# ===========================
# Створюємо словник для збереження моделей та результатів
models = {}
reports = {}

# Створюємо цикл для навчання моделі на:
# - оригінальних даних
# - даних після SMOTE
# - даних після SMOTE-Tomek
for name, (X_tr, y_tr) in {
    'Original': (X_train, y_train),
    'SMOTE': (X_train_smote, y_train_smote),
    'SMOTE-Tomek': (X_train_smote_tomek, y_train_smote_tomek)
}.items():
    # Ініціалізуємо класифікатор One-vs-Rest з Logistic Regression
    clf = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
    # Навчаємо модель
    clf.fit(X_tr, y_tr)
    # Робимо передбачення на тестовій вибірці
    y_pred = clf.predict(X_test)
    # Формуємо classification report як словник
    report = classification_report(y_test, y_pred, output_dict=True)
    # Обчислюємо f1_macro для зручного порівняння
    f1_macro = report['macro avg']['f1-score']
    reports[name] = (report, f1_macro)
    # Виводимо текстовий звіт
    print(f"\n=== {name} Dataset ===")
    print(classification_report(y_test, y_pred))

# ===========================
# 5. Порівняння моделей за F1 Macro
# ===========================
print("\n=== F1 Macro Scores ===")
for name, (_, f1_macro) in reports.items():
    print(f"{name}: F1 Macro = {f1_macro:.4f}")


=== Original Dataset ===
              precision    recall  f1-score   support

           0       0.39      0.39      0.39       394
           1       0.40      0.08      0.14       372
           2       0.47      0.63      0.54       394
           3       0.59      0.79      0.67       454

    accuracy                           0.49      1614
   macro avg       0.46      0.47      0.43      1614
weighted avg       0.47      0.49      0.45      1614


=== SMOTE Dataset ===
              precision    recall  f1-score   support

           0       0.41      0.39      0.40       394
           1       0.36      0.15      0.21       372
           2       0.48      0.62      0.54       394
           3       0.62      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.47      0.48      0.46      1614
weighted avg       0.47      0.50      0.47      1614


=== SMOTE-Tomek Dataset ===
              precision    recall  f1-score   suppor

Я завантажив підготовлені дані, збалансував їх за допомогою SMOTENC та SMOTE-Tomek, навчив моделі логістичної регресії у схемі One-vs-Rest для кожного з варіантів, отримав та проаналізував classification_report для кожної моделі, а також порівняв їх за метрикою f1_macro, яка є особливо корисною для багатокласових задач з можливим дисбалансом класів.